In [2]:
#sc.addPyFile("../graphframes-0.5.0-spark2.1-s_2.11.jar")

In [1]:
from graphframes import *
from pyspark.sql.functions import *

In [2]:
# Vertics DataFrame
v = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 37),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 38),
  ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend"),
  ("g", "e", "follow")
], ["src", "dst", "relationship"])

# Create a GraphFrame
g = GraphFrame(v, e)

g.vertices.show()
g.edges.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 37|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 38|
|  g|  Gabby| 60|
+---+-------+---+

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
|  g|  e|      follow|
+---+---+------------+



In [3]:
# g.vertices and g.edges are just DataFrames
# You can use any DataFrame API on them

g.edges.filter("src = 'a'").show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  a|  e|      friend|
+---+---+------------+



In [16]:
chain = g.find("(a)-[e1]->(b)")
followTo1 = lambda e: when(e['relationship'] == 'follow', 1).otherwise(0)
followed = chain.select('b', followTo1(chain['e1']).alias('f1') ).where( "f1==1 ").distinct()
name_fed = followed.select('b.name')

name_fed.show()

allname = v.select('name')
allname.show()
no_followed = allname.subtract(name_fed)
no_followed.show()

+-------+
|   name|
+-------+
| Esther|
|    Bob|
|  Fanny|
|Charlie|
+-------+

+-------+
|   name|
+-------+
|  Alice|
|    Bob|
|Charlie|
|  David|
| Esther|
|  Fanny|
|  Gabby|
+-------+

+-----+
| name|
+-----+
|Gabby|
|Alice|
|David|
+-----+



In [56]:
chain2 = g.find("(a)-[e1]->(b)")
followTo1 = lambda e: when(e['relationship'] == 'follow', 1).otherwise(0)
followed1 = chain.select('a', followTo1(chain['e1']).alias('f1') ).where( "f1==1 ").groupBy('a').count()
followed1=followed1.select('a.name', 'count')
followed1.show()

final= followed1.join(no_followed,'name')
#maxc = final.groupBy().max("count").take(1)[0][0]
#print(max)

max_num=final.groupBy().max("count").take(1)[0][0]
final.show()
final.filter(final.count == max_num).show()

#select('name').show()
##final=final.filter("'count'== maxc").select('name')

#final.show()
#les = g.find("(a)-[]->(b); (b)-[]->(c)")
#names = triangles.filter("a.name == 'Alice' and a.id != b.id AND a.id != c.id and b.id != c.id").select('c.name')
#names.show()

+-------+-----+
|   name|count|
+-------+-----+
|  Gabby|    1|
| Esther|    1|
|    Bob|    1|
|  Fanny|    1|
|Charlie|    1|
+-------+-----+

+-----+-----+
| name|count|
+-----+-----+
|Gabby|    1|
+-----+-----+



TypeError: condition should be string or Column

In [59]:
chain = g.find("(a)-[e1]->(b)")
followTo1 = lambda e: when(e['relationship'] == 'follow', 1).otherwise(0)
followed = chain.select('b', followTo1(chain['e1']).alias('f1') ).where( "f1==1 ").distinct()
name_fed = followed.select('b.name')

#name_fed.show()

allname = v.select('name')
#allname.show()
no_followed = allname.subtract(name_fed)
#no_followed.show()

chain2 = g.find("(a)-[e1]->(b)")
followTo1 = lambda e: when(e['relationship'] == 'follow', 1).otherwise(0)
followed1 = chain.select('a', followTo1(chain['e1']).alias('f1') ).where( "f1==1 ").groupBy('a').count()
followed1=followed1.select('a.name', 'count')
#followed1.show()

final= followed1.join(no_followed,'name')
#maxc = final.groupBy().max("count").take(1)[0][0]
#print(max)

max_num=final.groupBy().max("count").take(1)[0][0]
final.select('name').show()
#final.filter(final.count == max_num).show()

+-----+
| name|
+-----+
|Gabby|
+-----+



In [61]:
import math
def IsTrangleOrArea(x1,y1,x2,y2,x3,y3):
    return abs((x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2)) / 2.0)
 
def IsInside(x1,y1,x2,y2,x3,y3,x,y):
 
    #三角形ABC的面积
    ABC = IsTrangleOrArea(x1,y1,x2,y2,x3,y3)
 
    # 三角形PBC的面积
    PBC = IsTrangleOrArea(x,y,x2,y2,x3,y3)
 
    # 三角形ABC的面积
    PAC = IsTrangleOrArea(x1,y1,x,y,x3,y3)
 
    # 三角形ABC的面积
    PAB = IsTrangleOrArea(x1,y1,x2,y2,x,y)
 
    return (ABC == PBC + PAC + PAB)

IsInside(10, 30, 20, 0, 10, 30, 10, 15)

Py4JError: An error occurred while calling z:org.apache.spark.sql.functions.abs. Trace:
py4j.Py4JException: Method abs([class java.lang.Double]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:339)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:745)



In [13]:
allname = v.select('name').show()

+-------+
|   name|
+-------+
|  Alice|
|    Bob|
|Charlie|
|  David|
| Esther|
|  Fanny|
|  Gabby|
+-------+



In [7]:
g.edges.filter("src = 'a'").count()

2

In [8]:
# Count the number of followers of c.
# This queries the edge DataFrame.
print (g.edges.filter("relationship = 'follow' and dst = 'c'").count())

2


In [9]:
# A GraphFrame has additional attributes

g.outDegrees.show()

+---+---------+
| id|outDegree|
+---+---------+
|  g|        1|
|  f|        1|
|  e|        2|
|  d|        1|
|  c|        1|
|  b|        1|
|  a|        2|
+---+---------+



In [10]:
g.inDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  f|       1|
|  e|       2|
|  d|       1|
|  c|       2|
|  b|       2|
|  a|       1|
+---+--------+



In [11]:
g.inDegrees.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[dst#53], functions=[count(1)])
+- Exchange hashpartitioning(dst#53, 200)
   +- *(1) HashAggregate(keys=[dst#53], functions=[partial_count(1)])
      +- *(1) Project [dst#53]
         +- Scan ExistingRDD[src#52,dst#53,relationship#54]


In [12]:
myInDegrees = g.edges.groupBy('dst').count()\
               .withColumnRenamed('dst', 'id').withColumnRenamed('count', 'inDegree')
myInDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  f|       1|
|  e|       2|
|  d|       1|
|  c|       2|
|  b|       2|
|  a|       1|
+---+--------+



In [13]:
myInDegrees.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[dst#53], functions=[count(1)])
+- Exchange hashpartitioning(dst#53, 200)
   +- *(1) HashAggregate(keys=[dst#53], functions=[partial_count(1)])
      +- *(1) Project [dst#53]
         +- Scan ExistingRDD[src#52,dst#53,relationship#54]


In [14]:
print (g.inDegrees.storageLevel)

Serialized 1x Replicated


In [15]:
g.inDegrees.cache()

DataFrame[id: string, inDegree: int]

In [16]:
print (g.inDegrees.storageLevel)

Disk Memory Deserialized 1x Replicated


In [17]:
print (g.vertices.storageLevel)

Serialized 1x Replicated


In [18]:
g.cache()

GraphFrame(v:[id: string, name: string ... 1 more field], e:[src: string, dst: string ... 1 more field])

In [19]:
print (g.vertices.storageLevel)
print (g.edges.storageLevel)

Disk Memory Deserialized 1x Replicated
Disk Memory Deserialized 1x Replicated


In [20]:
# A triplet view of the graph

g.triplets.show()

+----------------+--------------+----------------+
|             src|          edge|             dst|
+----------------+--------------+----------------+
| [e, Esther, 32]|[e, f, follow]|  [f, Fanny, 38]|
|  [g, Gabby, 60]|[g, e, follow]| [e, Esther, 32]|
|  [a, Alice, 34]|[a, e, friend]| [e, Esther, 32]|
| [e, Esther, 32]|[e, d, friend]|  [d, David, 29]|
|  [f, Fanny, 38]|[f, c, follow]|[c, Charlie, 37]|
|    [b, Bob, 36]|[b, c, follow]|[c, Charlie, 37]|
|[c, Charlie, 37]|[c, b, follow]|    [b, Bob, 36]|
|  [a, Alice, 34]|[a, b, friend]|    [b, Bob, 36]|
|  [d, David, 29]|[d, a, friend]|  [a, Alice, 34]|
+----------------+--------------+----------------+



In [21]:
g.triplets.explain()

== Physical Plan ==
*(3) Project [src#231, edge#229, dst#233]
+- *(3) BroadcastHashJoin [edge#229.dst], [dst#233.id], Inner, BuildRight
   :- *(3) BroadcastHashJoin [edge#229.src], [src#231.id], Inner, BuildRight
   :  :- *(3) Project [named_struct(src, src#52, dst, dst#53, relationship, relationship#54) AS edge#229]
   :  :  +- InMemoryTableScan [dst#53, relationship#54, src#52]
   :  :        +- InMemoryRelation [src#52, dst#53, relationship#54], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
   :  :              +- Scan ExistingRDD[src#52,dst#53,relationship#54]
   :  +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, struct<id:string,name:string,age:bigint>, false].id))
   :     +- *(1) Project [named_struct(id, id#46, name, name#47, age, age#48L) AS src#231]
   :        +- InMemoryTableScan [age#48L, id#46, name#47]
   :              +- InMemoryRelation [id#46, name#47, age#48L], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
   : 

### Motif Finding

In [22]:
# Search for pairs of vertices with edges in both directions between them.
motifs = g.find("(a)-[]->(b); (b)-[]->(a)")#.filter('a.id > b.id')
motifs.show()

+----------------+----------------+
|               a|               b|
+----------------+----------------+
|    [b, Bob, 36]|[c, Charlie, 37]|
|[c, Charlie, 37]|    [b, Bob, 36]|
+----------------+----------------+



In [23]:
# Find triangles

triangles = g.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(a)")
triangles = triangles.filter("a.id < b.id AND a.id < c.id")
triangles.show()

+--------------+---------------+--------------+
|             a|              b|             c|
+--------------+---------------+--------------+
|[a, Alice, 34]|[e, Esther, 32]|[d, David, 29]|
+--------------+---------------+--------------+



In [3]:
triangles = g.find("(a)-[]->(b); (b)-[]->(c)")
names = triangles.filter("a.name == 'Alice' and a.id != b.id AND a.id != c.id and b.id != c.id").select('c.name')
names.show()

NameError: name 'g' is not defined

In [8]:

triangles = g.find("(a)-[]->(b); (b)-[]->(c)")
names = triangles.filter("a.name == 'Alice' and a.id != b.id AND a.id != c.id and b.id != c.id").select('c.name')
triangles2 = g.find("(a)-[]->(b)")
names2 = triangles2.filter("b.name == 'Alice'").select('a.name')
names.subtract(names2).show()

+-------+
|   name|
+-------+
|  Fanny|
|Charlie|
+-------+



In [24]:
triangles.explain()

== Physical Plan ==
*(6) Project [a#474, b#476, c#501]
+- *(6) BroadcastHashJoin [c#501.id, a#474.id], [__tmp-6526019406657860729#531.src, __tmp-6526019406657860729#531.dst], Inner, BuildRight
   :- *(6) Project [a#474, b#476, c#501]
   :  +- *(6) BroadcastHashJoin [__tmp-430217833014886237#499.dst], [c#501.id], Inner, BuildRight, (a#474.id < c#501.id)
   :     :- *(6) BroadcastHashJoin [b#476.id], [__tmp-430217833014886237#499.src], Inner, BuildRight
   :     :  :- *(6) Project [a#474, b#476]
   :     :  :  +- *(6) BroadcastHashJoin [__tmp-1043886091038848698#472.dst], [b#476.id], Inner, BuildRight, (a#474.id < b#476.id)
   :     :  :     :- *(6) BroadcastHashJoin [__tmp-1043886091038848698#472.src], [a#474.id], Inner, BuildRight
   :     :  :     :  :- *(6) Project [named_struct(src, src#52, dst, dst#53, relationship, relationship#54) AS __tmp-1043886091038848698#472]
   :     :  :     :  :  +- InMemoryTableScan [dst#53, relationship#54, src#52]
   :     :  :     :  :        +- InMem

In [25]:
# Negation
oneway = g.find("(a)-[]->(b); !(b)-[]->(a)")
oneway.show()

+---------------+----------------+
|              a|               b|
+---------------+----------------+
| [a, Alice, 34]| [e, Esther, 32]|
|[e, Esther, 32]|  [d, David, 29]|
| [a, Alice, 34]|    [b, Bob, 36]|
| [g, Gabby, 60]| [e, Esther, 32]|
|[e, Esther, 32]|  [f, Fanny, 38]|
| [f, Fanny, 38]|[c, Charlie, 37]|
| [d, David, 29]|  [a, Alice, 34]|
+---------------+----------------+



In [26]:
# Find vertices without incoming edges. This is wrong:
g.find('!()-[]->(a)').show()
# Because negation is implemented as a subtraction

Py4JJavaError: An error occurred while calling o101.find.
: org.graphframes.InvalidPatternException
	at org.graphframes.GraphFrame$.org$graphframes$GraphFrame$$findIncremental(GraphFrame.scala:892)
	at org.graphframes.GraphFrame.findSimple(GraphFrame.scala:441)
	at org.graphframes.GraphFrame.find(GraphFrame.scala:316)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.r

In [27]:
# Still doesn't work:
g.vertices.join(g.inDegrees, 'id').filter('inDegree=0').show()

+---+----+---+--------+
| id|name|age|inDegree|
+---+----+---+--------+
+---+----+---+--------+



In [28]:
# Why? Because inDegree is computed by a groupBy followed by a count
g.inDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  f|       1|
|  e|       2|
|  d|       1|
|  c|       2|
|  b|       2|
|  a|       1|
+---+--------+



In [29]:
# Correct way:
g.vertices.join(g.inDegrees, 'id', 'left_outer').filter('inDegree is null').show()

+---+-----+---+--------+
| id| name|age|inDegree|
+---+-----+---+--------+
|  g|Gabby| 60|    null|
+---+-----+---+--------+



In [30]:
# Or use subtract:
g.vertices.select('id').subtract(g.inDegrees.select('id')).join(g.vertices,'id').show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  g|Gabby| 60|
+---+-----+---+



In [31]:
# More meaningful queries can be expressed by applying filters.
# Question: where is this filter applied?

g.find("(a)-[]->(b); (b)-[]->(a)").filter("b.age > 36").show()

+------------+----------------+
|           a|               b|
+------------+----------------+
|[b, Bob, 36]|[c, Charlie, 37]|
+------------+----------------+



In [32]:
g.find("(a)-[]->(b); (b)-[]->(a)").filter("b.age > 36").explain()

== Physical Plan ==
*(4) Project [a#1329, b#1331]
+- *(4) BroadcastHashJoin [b#1331.id, a#1329.id], [__tmp-3851898762290097694#1354.src, __tmp-3851898762290097694#1354.dst], Inner, BuildRight
   :- *(4) Project [a#1329, b#1331]
   :  +- *(4) BroadcastHashJoin [__tmp-4480780508598698291#1327.dst], [b#1331.id], Inner, BuildRight
   :     :- *(4) BroadcastHashJoin [__tmp-4480780508598698291#1327.src], [a#1329.id], Inner, BuildRight
   :     :  :- *(4) Project [named_struct(src, src#52, dst, dst#53, relationship, relationship#54) AS __tmp-4480780508598698291#1327]
   :     :  :  +- InMemoryTableScan [dst#53, relationship#54, src#52]
   :     :  :        +- InMemoryRelation [src#52, dst#53, relationship#54], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
   :     :  :              +- Scan ExistingRDD[src#52,dst#53,relationship#54]
   :     :  +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, struct<id:string,name:string,age:bigint>, false].id))
   :     :    

In [33]:
chain4 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)")
chain4.show()

+----------------+--------------+----------------+--------------+----------------+--------------+----------------+
|               a|            e1|               b|            e2|               c|            e3|               d|
+----------------+--------------+----------------+--------------+----------------+--------------+----------------+
|  [a, Alice, 34]|[a, b, friend]|    [b, Bob, 36]|[b, c, follow]|[c, Charlie, 37]|[c, b, follow]|    [b, Bob, 36]|
|    [b, Bob, 36]|[b, c, follow]|[c, Charlie, 37]|[c, b, follow]|    [b, Bob, 36]|[b, c, follow]|[c, Charlie, 37]|
|[c, Charlie, 37]|[c, b, follow]|    [b, Bob, 36]|[b, c, follow]|[c, Charlie, 37]|[c, b, follow]|    [b, Bob, 36]|
|  [f, Fanny, 38]|[f, c, follow]|[c, Charlie, 37]|[c, b, follow]|    [b, Bob, 36]|[b, c, follow]|[c, Charlie, 37]|
| [e, Esther, 32]|[e, f, follow]|  [f, Fanny, 38]|[f, c, follow]|[c, Charlie, 37]|[c, b, follow]|    [b, Bob, 36]|
| [e, Esther, 32]|[e, d, friend]|  [d, David, 29]|[d, a, friend]|  [a, Alice, 34

In [77]:
# Find chains of 4 vertices such that at least 2 of the 3 edges are "friend" relationships.
# The when function is similar to the CASE WHEN in SQL

chain4 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)")

friendTo1 = lambda e: when(e['relationship'] == 'friend', 1).otherwise(0)

chain4.select('*',friendTo1(chain4['e1']).alias('f1'), \
                  friendTo1(chain4['e2']).alias('f2'), \
                  friendTo1(chain4['e3']).alias('f3')) \
      .where('f1 + f2 + f3 >= 2 AND a != d').select('a', 'b', 'c', 'd').show()

+---------------+---------------+---------------+----------------+
|              a|              b|              c|               d|
+---------------+---------------+---------------+----------------+
|[e, Esther, 32]| [d, David, 29]| [a, Alice, 34]|    [b, Bob, 36]|
| [d, David, 29]| [a, Alice, 34]|[e, Esther, 32]|  [f, Fanny, 38]|
| [d, David, 29]| [a, Alice, 34]|   [b, Bob, 36]|[c, Charlie, 37]|
| [g, Gabby, 60]|[e, Esther, 32]| [d, David, 29]|  [a, Alice, 34]|
+---------------+---------------+---------------+----------------+



In [83]:
chain = g.find("(a)-[e1]->(b)")
followTo1 = lambda e: when(e['relationship'] == 'follow', 1).otherwise(0)
chain.select('a', followTo1(chain['e1']).alias('f1') ).where( "f1==1 and b.name = 'Charlie'").select('a').show()

                      

+--------------+
|             a|
+--------------+
|  [b, Bob, 36]|
|[f, Fanny, 38]|
+--------------+



+----------------+---+
|               b| f1|
+----------------+---+
| [e, Esther, 32]|  1|
|    [b, Bob, 36]|  1|
|  [f, Fanny, 38]|  1|
|[c, Charlie, 37]|  1|
+----------------+---+



In [96]:
chain = g.find("(a)-[e1]->(b)")
followTo1 = lambda e: when(e['relationship'] == 'follow', 1).otherwise(0)
people = chain.select('b', followTo1(chain['e1']).alias('f1') ).where( "f1==1").groupBy('b').count()
people.filter('count > 1').select('b').show()
                      

+----------------+
|               b|
+----------------+
|[c, Charlie, 37]|
+----------------+



In [97]:
from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory. Required for stateful transformations
ssc.checkpoint("checkpoint")

numPartitions = 8
rdd = sc.textFile('adj_noun_pairs.txt', numPartitions)
rddQueue = rdd.randomSplit([1]*10, 123)
lines = ssc.queueStream(rddQueue)

# FILL IN YOUR CODE

ssc.start()
ssc.awaitTermination(50)
ssc.stop(False)
print("Finished")

IllegalArgumentException: 'requirement failed: No output operations registered, so nothing to execute'

### Subgraphs

In [35]:
# Select subgraph of users older than 30, and edges of type "friend"
v2 = g.vertices.filter("age > 30")
e2 = g.edges.filter("relationship = 'friend'")
g2 = GraphFrame(v2, e2)
g2.vertices.show()
g2.edges.show()

# GraphFrames does not check if a vertex is isolated (which is OK)
# or if an edge connects two existing vertices (which could cause bugs)

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 37|
|  e| Esther| 32|
|  f|  Fanny| 38|
|  g|  Gabby| 60|
+---+-------+---+

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
+---+---+------------+



In [36]:
g2.inDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  e|       1|
|  d|       1|
|  b|       1|
|  a|       1|
+---+--------+



In [37]:
# Only keeping edges that connect existing vertices
e3 = e2.join(v2, e2['src'] == v2['id'], 'left_semi') \
       .join(v2, e2['dst'] == v2['id'], 'left_semi') 
g3 = GraphFrame(v2, e3)

In [38]:
g3.edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  a|  e|      friend|
+---+---+------------+



In [39]:
# Select subgraph based on edges of type "follow"
# pointing from an older user to an youner user.
e4 = g.find("(a)-[e]->(b)")\
        .filter("e.relationship = 'follow'")\
        .filter("a.age > b.age") \
        .select("e.*")
e4.show()

# Only keeping vertices that appear in the edges
v4 = g.vertices.join(e4, g.vertices['id'] == e4['src'], 'leftsemi') \
      .union(g.vertices.join(e4, g.vertices['id'] == e4['dst'], 'leftsemi')) \
      .distinct()
    
# Construct the subgraph
g4 = GraphFrame(v4, e4)
g4.vertices.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  c|  b|      follow|
|  f|  c|      follow|
|  g|  e|      follow|
+---+---+------------+

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  g|  Gabby| 60|
|  e| Esther| 32|
|  b|    Bob| 36|
|  f|  Fanny| 38|
|  c|Charlie| 37|
+---+-------+---+



In [41]:
g4.triplets.show()

+----------------+--------------+----------------+
|             src|          edge|             dst|
+----------------+--------------+----------------+
|[c, Charlie, 37]|[c, b, follow]|    [b, Bob, 36]|
|  [f, Fanny, 38]|[f, c, follow]|[c, Charlie, 37]|
|  [g, Gabby, 60]|[g, e, follow]| [e, Esther, 32]|
+----------------+--------------+----------------+



### BFS

In [46]:
# Starting vertex is 'a'
layers = [g.vertices.select('id').where("id = 'a'")]
print(layers[0])
visited =  layers[0]

while layers[-1].count() > 0:
    # From the current layer, get all the one-hop neighbors
    d1 = layers[-1].join(g.edges, layers[-1]['id'] == g.edges['src'])
    # Rename the column as 'id', and remove visited verices and duplicates
    d2 = d1.select(d1['dst'].alias('id')) \
           .subtract(visited).distinct()
    layers += [d2]
    visited = visited.union(layers[-1])

DataFrame[id: string]


In [39]:
layers[0].show()

+---+
| id|
+---+
|  a|
+---+



In [40]:
layers[1].show()

+---+
| id|
+---+
|  e|
|  b|
+---+



In [68]:
layers[2].show()

+---+
| id|
+---+
|  f|
|  d|
|  c|
+---+



In [69]:
layers[3].show()

+---+
| id|
+---+
+---+



In [48]:
# GraphFrames provides own BFS:

paths = g.bfs("id = 'a'", "age > 36")
paths.show()

+--------------+--------------+---------------+--------------+----------------+
|          from|            e0|             v1|            e1|              to|
+--------------+--------------+---------------+--------------+----------------+
|[a, Alice, 34]|[a, b, friend]|   [b, Bob, 36]|[b, c, follow]|[c, Charlie, 37]|
|[a, Alice, 34]|[a, e, friend]|[e, Esther, 32]|[e, f, follow]|  [f, Fanny, 38]|
+--------------+--------------+---------------+--------------+----------------+



### List Ranking

In [34]:
# -1 denotes end of list
data = [(0, 5), (1, 0), (3, 4), (4, 6), (5, -1), (6,1)]
e = spark.createDataFrame(data, ['src', 'dst'])
v = e.select(col('src').alias('id'), when(e.dst == -1, 0).otherwise(1).alias('d'))
v1 = spark.createDataFrame([(-1, 0)], ['id', 'd'])
v = v.union(v1)
v.show()
e.show()

+---+---+
| id|  d|
+---+---+
|  0|  1|
|  1|  1|
|  3|  1|
|  4|  1|
|  5|  0|
|  6|  1|
| -1|  0|
+---+---+

+---+---+
|src|dst|
+---+---+
|  0|  5|
|  1|  0|
|  3|  4|
|  4|  6|
|  5| -1|
|  6|  1|
+---+---+



In [49]:
while e.filter('dst != -1').count() > 0:
    g = GraphFrame(v, e)
    g.cache()
    v = g.triplets.select(col('src.id').alias('id'), 
                          (col('src.d') + col('dst.d')).alias('d')) \
         .union(v1)
    e = g.find('(a)-[]->(b); (b)-[]->(c)') \
         .select(col('a.id').alias('src'), col('c.id').alias('dst')) \
         .union(e.filter('dst = -1'))
v.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 37|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 38|
|  g|  Gabby| 60|
+---+-------+---+

